In [37]:
#imports
import os, sys, json, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import glob as glob

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown


from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats, get_key, get_recent_file


In [38]:
print(state_ref)

# Create regex pattern to match any state name
state_pattern = re.compile(r'\b(' + '|'.join(state_ref.keys()) + r')\b', re.IGNORECASE)
def extract_state_abbreviation(text):
    """Finds state name in text and returns its abbreviation"""
    match = state_pattern.search(text)
    if match:
        state_name = match.group(0)  # Extract the matched state name
        return state_ref[state_name.title()]  # Convert to title case and look up abbreviation
    return None  # Return None if no state is found



{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'}


In [ ]:
os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\attendance\attendance data\output')
files = glob.glob(r'*.xlsx')
for file in files:
    df = pd.read_excel(file)
    # print(df.head().to_string())


In [41]:
#Insert Planning Doc File Here and File Name
file = r"c:\Users\clutz\Downloads\2025 Alaska Summit Logistics Planning Doc.xlsx"
filename = '2025_Alaska_Summit.xlsx'

# Data Imports

In [42]:

import warnings
warnings.simplefilter("ignore", UserWarning)


# hlr_planning_doc = pd.read_excel(file)

hlr_planning_doc  = pd.read_excel(open(file, 'rb'), sheet_name=None)

for k,v in hlr_planning_doc.items():
    # if 'site' in str(k).lower().strip():
    if re.search(r'[Oo]n(\s+)?(-)?[Ss]ite', str(k)):
        event_attendance = v
        # break
        # print(k)
    if re.search(r'[Aa]ttendance(\s+)?[Tt]racker', str(k)):
        attendee_info = v


# Attendance Tracking

In [ ]:
#get start row and cut df
for row in event_attendance.itertuples(index=True):
    # print(len(row))
    # print( type(row))
    # print(row.values)
    row_string = "|".join(map(str, row))
    # print(row_string)

    if re.search(r'[Hh]onorific', row_string):
        # print(row.Index)
        # print(row_string)
        start_index = row.Index
        continue


#implement row narrowing
event_data_cl = event_attendance.iloc[start_index:,:].reset_index(drop=True)
event_data_cl.columns = event_data_cl.iloc[0]
event_data_cl = event_data_cl.iloc[1:,:]


#get col subset
for name,value in event_data_cl.items():
    print(name)
    if re.search(r'[Ll]ast', str(name)):
        stop_col = name
        print(f'stop_col: {stop_col}')
        break

#implement col narrowing
attendance_tracking_df = event_data_cl.loc[:,:stop_col]
attendance_tracking_df


Status 
Attendee Type
Honorific
First
Last
stop_col: Last


,Status,Attendee Type,Honorific,First,Last
1,Not checked in,Invited Guests,Ms.,Tamar,Ben-Yosef
2,Not checked in,Legislator,0,Stephanie,Berglund
3,Not checked in,Legislator,0,Mary Kathryn,Brewer
4,Not checked in,Legislator,Representative,Ashley,Carrick
5,Not checked in,Legislator,Senator,Matt,Claman
...,...,...,...,...,...
68,Not checked in,NaN,NaN,NaN,NaN
69,Not checked in,NaN,NaN,NaN,NaN
70,Not checked in,NaN,NaN,NaN,NaN
71,Not checked in,NaN,NaN,NaN,NaN


In [44]:
#find cols and filter
cols_found = []
for col in attendance_tracking_df:
    # print(col)
    if re.search('[Ss]tatus', str(col)):
        status_col = str(col)
        cols_found.append(col)
    elif re.search(r'[Aa]ttendee(\s+)?[Tt]ype', str(col)):
        att_type_col = str(col)
        cols_found.append(col)


attendance_cl = attendance_tracking_df[
    attendance_tracking_df[status_col].fillna('').str.contains('Checked', na=False) & 
    ~(attendance_tracking_df[att_type_col].fillna('').str.strip().str.lower() == "thi staff")  # Using ~ (negation)
]
attendance_cl


,Status,Attendee Type,Honorific,First,Last


# Attendee Info

In [45]:
#get start row and cut df
for row in attendee_info.itertuples(index=True):
    # print(len(row))
    # print( type(row))
    # print(row.values)
    row_string = "|".join(map(str, row))
    # print(row_string)

    if re.search(r'[Hh]onorific', row_string):
        # print(row.Index)
        # print(row_string)
        start_index = row.Index
        break

#implement row bounds
attendee_info_cl = attendee_info.iloc[start_index:,:].reset_index(drop=True)
attendee_info_cl.columns = attendee_info_cl.iloc[0]
attendee_info_cl.columns = [str(x).strip() for x in attendee_info_cl.columns]
attendee_info_cl = attendee_info_cl.iloc[1:,:].reset_index(drop=True)

start_col = np.nan
stop_col = np.nan
#get col subset
for name,value in attendee_info_cl.items():
    print(name)
    if re.search(r'[Ff]irst', str(name)):
        start_col = name
        # print(f'start_col: {start_col}')
        continue

    elif re.search(r'[Oo]rg', str(name)):
        stop_col = name
        # print(f'stop_col: {stop_col}')
        break


# print(f'start col:"{start_col}"')
# print(f'stop col:"{stop_col}"')
# print(type)
# print(attendee_info_cl.columns)
# print(attendee_info_cl[start_col])
#implement col bounds
attendee_info_df = attendee_info_cl.loc[:,start_col:stop_col]
attendee_info_df = attendee_info_df.dropna(how='all', axis=0)


Attendee Type / Role
Registration Date
Covered by THI or Self Pay
Registration Status
nan
Honorific
First Name
Last Name
Title
Organization (State for HKF and District # for SLR)


# Both

In [46]:
#edit col names
attendance_cl.columns = [re.sub(r"[Nn]ame|\(.+\)", "", str(x)).lower().strip() for x in attendance_cl.columns]
attendee_info_df.columns = [re.sub(r"[Nn]ame|\(.+\)", "", str(x)).lower().strip() for x in attendee_info_df.columns]


In [47]:
attendance_cl.columns

Index(['status', 'attendee type', 'honorific', 'first', 'last'], dtype='object')

In [48]:
attendee_info_df['full'] = attendee_info_df['first'] + " " + attendee_info_df['last']
attendee_info_df = attendee_info_df.drop(columns=['first', 'last'], axis = 1)
attendance_cl['full'] = attendance_cl['first'] + " " + attendance_cl['last']

In [49]:
#merge and clean data/add additional columns
#merge data
final_df = attendance_cl.merge(attendee_info_df,on='full' , how='left').reset_index(drop=True)
final_df = final_df.drop('full', axis=1)
#clean
final_df = final_df.drop('status', axis = 1)
final_df.columns

#fill in state where available
final_df['state'] = final_df['organization'].apply(extract_state_abbreviation)

short_convening_val = False
final_df['short_convening'] = short_convening_val


final_df

,attendee type,honorific,first,last,title,organization,state,short_convening


In [27]:
#pull in template and get col mapping
template = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\attendance\attendance data\to be processed\2025_GAR_attendance.xlsx"
template_df = pd.read_excel(template)
print(template_df.columns)
col_mapping = {}
#go through template
for col in template_df.columns:
    #get col
    search_string = col.replace('_name','').lower()
    
    #look for col that matchs
    #for each col look for it in the merged_df cols
    for m in final_df.columns:
        #return found value   
        # print(m)
        if re.search(rf'{m.lower()}', search_string):
            print(f'mapping {m} to {col}')
            col_mapping[m] = col
            break

        elif re.search(r'[Aa]ttendee\s[Tt]ype|[Rr]ole', search_string):
            print(f'mapping {m} to {col}')
            col_mapping[m] = col
            break
        elif re.search(r'^[Oo]rg', search_string) and re.search(r'^[Oo]rg', str(m)):
            print(f'mapping {m} to {col}')
            col_mapping[m] = col
            break

        

            
# list


Index(['first_name', 'last_name', 'honorific', 'title', 'org', 'district',
       'role', 'state', 'short_convening'],
      dtype='object')
mapping first to first_name
mapping last to last_name
mapping honorific to honorific
mapping title to title
mapping organization to org
mapping attendee type to role
mapping state to state
mapping short_convening to short_convening


In [28]:
#print out of unassigned and unaccessed
#not assigned
n_assigned_to_dict = [x for x in template_df.columns if x not in col_mapping.values()]

#not pulled
n_accessed = [x for x in final_df.columns if x not in col_mapping.keys()]

print(f'not assigned: {n_assigned_to_dict}')
print(f'not accessed: {n_accessed}')

not assigned: ['district']
not accessed: []


In [29]:
#ONLY RUN IF YOU ARE DONE: assigns np.nan to any columns not included per template
for n in n_assigned_to_dict:
    final_df[n] = np.nan

In [30]:
#remap column names
final_df_remapped = final_df.rename(columns=col_mapping).loc[:,list(template_df.columns)]


# Export

In [32]:
os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\attendance\attendance data\output')
final_df_remapped.to_excel(filename, index=False)